# ML Zoomcamp 2023 – Second Capstone Project

[Stanford Dogs Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs)

In [2]:
import numpy as np
import os, PIL
from PIL import Image
import tensorflow as tf